# Descriptive Statistics and the Importance of Weighting

We are now going to work the the 'micro' data directly from ADH. Luckily, some of our coding friends at NYU Stern have done a tonne of the heavy lifting for us and merged all of ADH's essential data together into one file.

The trickiest thing to understand is the timing of the data and the variable names.

Here is some info:

#### Details about timing is as follows. 
The start of the period is 1991 and then end is 2007. 
This is then divided into two periods. 
The first periods is 1991-2000, thus this is a 9 year time period. 
They convert stuff into a "comparable decadal scale" see Footnote 22. 
Thus, for values for this period, they multiply them by 10/9.
The same issue arises for the second period which is 2000-2007. 
The values for this are again converted to "decadal scales" so they are multiplied by 10/7.

The Appendix Table 2, reports the income variable and the decadal adjustments. 
In the summary statistics for the stuff that we care about, the ADH data is adjusted in this way described above.

That is, variables starting with 'l' are in levels whereas variables starting with 'd' are the decadal equivalents.

As necessary, we will tell you which variable to use, so that $*_somevariable$ means to choose the appropriate level or decadal equivalent. We leave you to figure out which of the $l$ or $d$ variables to use. Do ask us if you are confused.

## Understanding the Essence of the Paper and What Comes Next.

Read Section 1 of ADH, so that you build an understanding of there main measure 'IPW' and what the paper is about. This will help you understand the context behind the remaining exercises in this notebook and those to follow.

Your first task will be to compute some descriptive statistics from the data. To be more precise, you will replicate some of the key numbers in Appendix Table 2 of ADH. (On a side note, at least one of us thinks this table should be in the main text!)

## Loading the Necessary packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Downloading the ADH data.

The data are included inside the larger folder we are working in. If your current working directory is 'notebooks', then the data are stored in '../data'

Load this data into a data frame called `df` using the appropriate pandas `read_*` function. Then look at the head and tail of the data.

In [4]:
data_file = '../data/adh_data.csv'

In [5]:
df = pd.read_csv(data_file)

In [6]:
df.head()

,czone,statefip,city,yr,t2,timepwt48,reg_midatl,reg_encen,reg_wncen,reg_satl,...,d_tradefactor_otch_lag_io,d_expfactor_otch_lag_io,d_tradeuschlw_pw,d_tradeotchlw_pw_lag,d_tradeuschce_pw,d_tradeotchce_pw_lag,d_tradeusce_pw,d_tradeotce_pw_lag,d_tradeushi_pw,d_tradeothi_pw_lag
0,100,47,undefined,1990,0,0.002114,0,0,0,0,...,2.236522,0.473667,5.745085,2.437189,11.723440,2.363754,6.429653,0.084927,30.744575,8.696790
1,100,47,undefined,2000,1,0.002067,0,0,0,0,...,7.756201,2.385448,7.433516,9.709133,10.229408,9.147025,3.609548,0.178625,25.120905,64.122138
2,200,47,undefined,1990,0,0.000732,0,0,0,0,...,2.819397,0.470286,3.139381,3.033377,5.698626,2.847327,2.668147,0.049593,16.205678,3.316682
3,200,47,undefined,2000,1,0.000815,0,0,0,0,...,4.411247,0.982626,10.840832,4.853496,13.314946,4.596729,3.054200,0.115029,14.212564,21.742845
4,301,47,undefined,1990,0,0.000261,0,0,0,0,...,1.100026,0.110491,2.656860,0.798065,6.746723,0.739753,4.684127,0.018845,18.276355,0.465902


## Summary Statistics by Year - Take 1

Let's try and build up some understanding of the data.

Compute the for each decade the number of chinese imports per worker in levels ('l_tradeusch_pw'):

In [7]:
df.groupby('yr')['l_tradeusch_pw'] \
        .aggregate(mean  = np.mean)


,mean
yr,
1990,0.363711
2000,1.123239


and per decadal equivalent

In [8]:
df.groupby('yr')['d_tradeusch_pw'] \
        .aggregate(mean  = np.mean)

,mean
yr,
1990,1.175688
2000,2.635525


Notice that these statistics do not line up exactly with those in Appendix Table 2. The reason why? 

They use *weighted* statistics, and we haven't. 

There is a new-ish package that can help us here (unfortunately pandas alone doesn't do us many favours). The package we need to install is called `weightedcalcs`


In [9]:
!pip install weightedcalcs

In [10]:
import weightedcalcs as wc

weightedcalcs works well on pandas dataframes, though the exact syntax differs a little. It's useful to know - especially if you want to work with census data.

First, we create a `calc` object that specifies what we want to weight by.

ADH weight by the level of the population count per region, `l_popcount`:


In [11]:
calc = wc.Calculator("l_popcount")

This operator cannot be chained into an existing pandas command as far as I am aware. What it can do though, is take a grouped dataframe as an input argument, so the syntax:

```python
my_grouped_df = df.groupby('some_grouping_variable')

calc.SOMESTAT(my_grouped_df, 'my_intersting_variable')
```

works just fine.

So:

In [12]:
grp_df = df.groupby('yr')

In [13]:
calc.mean(grp_df, "l_tradeusch_pw").round(2)

yr
1990    0.29
2000    1.08
dtype: float64

In [14]:
calc.mean(grp_df, "d_tradeusch_pw").round(2)

yr
1990    1.14
2000    2.63
dtype: float64

yields exactly the numbers in appendix table 2 (thou you need to look for exactly where the numbers appear).

Let's do some more weighted group stats.

The appendix table also reports standard deviations. Let's try this too, the relevant function is `std`. Give it a go:

In [15]:
calc.std(grp_df, "l_tradeusch_pw").round(2)

yr
1990    0.32
2000    0.90
dtype: float64

In [16]:
calc.std(grp_df, "d_tradeusch_pw").round(2)

yr
1990    0.99
2000    2.01
dtype: float64

Now compute the mean and standard deviation of the average household wage and salary, `*_avg_hhincwage_pc_pw`:

In [17]:
calc.mean(grp_df, "l_avg_hhincwage_pc_pw").round(2)

yr
1990    23495.65
2000    27654.94
dtype: float64

In [18]:
calc.std(grp_df, "l_avg_hhincwage_pc_pw").round(2)

yr
1990    4700.44
2000    5449.15
dtype: float64

In [19]:
calc.mean(grp_df, "d_avg_hhincwage_pc_pw").round(2)

yr
1990    4152.03
2000    1703.21
dtype: float64

In [20]:
calc.std(grp_df, "d_avg_hhincwage_pc_pw").round(2)

yr
1990    1569.38
2000    2623.23
dtype: float64

And once more for share not in labor force, `*_sh_nilf`

In [21]:
calc.mean(grp_df, "l_sh_nilf").round(2)

yr
1990    24.76
2000    26.05
dtype: float64

In [22]:
calc.std(grp_df, "l_sh_nilf").round(2)

yr
1990    4.34
2000    4.39
dtype: float64

In [23]:
calc.mean(grp_df, "d_sh_nilf").round(2)

yr
1990    1.29
2000   -1.82
dtype: float64

In [24]:
calc.std(grp_df, "d_sh_nilf").round(2)

yr
1990    2.56
2000    2.57
dtype: float64

How well do your statistics line up with those reported in the paper?